In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("project.db")

gdp_pc = pd.read_csv("../data/raw/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_2624.csv", skiprows=4)
gdp_growth = pd.read_csv("../data/raw/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_2509.csv", skiprows=4)
emp = pd.read_csv("../data/raw/API_SL.EMP.TOTL.SP.ZS_DS2_en_csv_v2_4653.csv", skiprows=4)

gdp_pc.to_sql("gdp_per_capita_raw", conn, if_exists="replace", index=False)
gdp_growth.to_sql("gdp_growth_raw", conn, if_exists="replace", index=False)
emp.to_sql("employment_raw", conn, if_exists="replace", index=False)

print("Tables written to SQLite!")

#code by Justin


Tables written to SQLite!


In [3]:
import pandas as pd
import sqlite3


conn = sqlite3.connect("project.db")


gdp_pc = pd.read_sql("SELECT * FROM gdp_per_capita_raw", conn)
gdp_growth = pd.read_sql("SELECT * FROM gdp_growth_raw", conn)
emp = pd.read_sql("SELECT * FROM employment_raw", conn)


def clean_wdi(df):
    """turn WDI format into tidy format：Country, Year, Value"""
    df = df.melt(
        id_vars=["Country Name", "Country Code"],
        var_name="Year",
        value_name="Value"
    )
    
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

    df = df.dropna(subset=["Year"])
    return df

gdp_pc_clean = clean_wdi(gdp_pc)
gdp_growth_clean = clean_wdi(gdp_growth)
emp_clean = clean_wdi(emp)


countries = ["United States", "China"]
year_filter = (gdp_pc_clean["Year"] >= 1990) & (gdp_pc_clean["Year"] <= 2023)

gdp_pc_clean = gdp_pc_clean[gdp_pc_clean["Country Name"].isin(countries) & year_filter]
gdp_growth_clean = gdp_growth_clean[gdp_growth_clean["Country Name"].isin(countries)]
emp_clean = emp_clean[emp_clean["Country Name"].isin(countries)]


gdp_pc_clean.to_sql("gdp_per_capita_clean", conn, if_exists="replace", index=False)
gdp_growth_clean.to_sql("gdp_growth_clean", conn, if_exists="replace", index=False)
emp_clean.to_sql("employment_clean", conn, if_exists="replace", index=False)

print("Clean tables written to SQLite!")

# code by Justin


Clean tables written to SQLite!


In [5]:

gdp_pc_clean = pd.read_sql("SELECT * FROM gdp_per_capita_clean", conn)
gdp_growth_clean = pd.read_sql("SELECT * FROM gdp_growth_clean", conn)
emp_clean = pd.read_sql("SELECT * FROM employment_clean", conn)


gdp_pc_clean.to_csv("../data/processed/gdp_per_capita_clean.csv", index=False)
gdp_growth_clean.to_csv("../data/processed/gdp_growth_clean.csv", index=False)
emp_clean.to_csv("../data/processed/employment_clean.csv", index=False)

print("Processed CSV files saved!")

#code by Justin


Processed CSV files saved!
